<a href="https://colab.research.google.com/github/Martinez301105/ED-2025-1/blob/main/Paquete_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import math
from tabulate import tabulate

# =============================================
# 1. Sistemas de ecuaciones no lineales (Broyden)
# =============================================

def broyden_method(F, x0, max_iter, tol):
    """Implementación del método de Broyden"""
    n = len(x0)
    B = np.eye(n)
    x = x0.copy()
    iteraciones = []

    for k in range(max_iter):
        Fx = F(x)
        iteraciones.append([k] + list(x) + [np.linalg.norm(Fx)])

        if np.linalg.norm(Fx) < tol:
            return x, True, iteraciones

        dx = np.linalg.solve(B, -Fx)
        x_new = x + dx
        Fx_new = F(x_new)
        y = Fx_new - Fx
        B = B + np.outer((y - B @ dx), dx) / (dx @ dx)
        x = x_new

    return x, False, iteraciones

def ejecutar_broyden():
    """Interfaz para el método de Broyden"""
    sistemas = [
        {
            "nombre": "Sistema 1",
            "F": lambda x: np.array([
                x[0]**2 + x[0]*x[1] + 2*x[1]**2 - 5,
                5*x[1] - 2*x[0]*x[1]**2 + 3
            ]),
            "n": 2,
            "desc": """Ecuaciones:
            f1(x,y) = x² + xy + 2y² - 5 = 0
            f2(x,y) = 5y - 2xy² + 3 = 0"""
        },
        {
            "nombre": "Sistema 2",
            "F": lambda x: np.array([
                x[0]**2 - 3*x[1]**2 - 10,
                2*x[1]**2 - 3*x[0]*x[1] + 1
            ]),
            "n": 2,
            "desc": """Ecuaciones:
            f1(x,y) = x² - 3y² - 10 = 0
            f2(x,y) = 2y² - 3xy + 1 = 0"""
        }
    ]

    print("\n=== Método de Broyden para Sistemas No Lineales ===")
    print("Sistemas disponibles:")
    for i, sistema in enumerate(sistemas, 1):
        print(f"{i}. {sistema['nombre']}")
    print("3. Volver al menú principal")

    try:
        opcion = int(input("\nSeleccione un sistema (1-2) o 3 para volver: "))
        if opcion == 3:
            return
        sistema = sistemas[opcion-1]
    except (ValueError, IndexError):
        print("Opción no válida.")
        return

    print(f"\n{sistema['desc']}")

    while True:
        print(f"\nIngrese {sistema['n']} valores iniciales:")
        try:
            x0 = np.array(list(map(float, input("Valores iniciales (separados por espacios): ").split())))
            if len(x0) != sistema["n"]:
                print(f"Error: Debe ingresar {sistema['n']} valores.")
                continue

            max_iter = int(input("Número máximo de iteraciones: "))
            tol = float(input("Tolerancia: "))

            sol, converged, iteraciones = broyden_method(sistema["F"], x0, max_iter, tol)

            headers = ["Iter"] + [f"x{i+1}" for i in range(sistema["n"])] + ["Error"]
            print("\nResultados:")
            print(tabulate(iteraciones, headers=headers, tablefmt="grid", floatfmt=".6f"))

            if converged:
                print(f"\nSolución encontrada: {sol}")
            else:
                print("\nNo convergió en las iteraciones dadas.")

        except ValueError:
            print("Error: Entrada inválida.")

        if input("\n¿Intentar con otros valores iniciales? (s/n): ").lower() != 's':
            break

# =============================================
# 2. Interpolación y ajuste de curvas
# =============================================

# Función común para capturar datos
datos_compartidos = []

def capturar_datos():
    """Captura y valida datos para interpolación/ajuste"""
    global datos_compartidos
    datos = []

    try:
        n = int(input("\nNúmero de puntos (mínimo 2): "))
        if n < 2:
            print("Se necesitan al menos 2 puntos.")
            return None
    except ValueError:
        print("Debe ingresar un número entero.")
        return None

    print("\nIngrese los puntos (x, y):")
    for i in range(n):
        while True:
            try:
                x = float(input(f"x[{i}] = "))
                y = float(input(f"y[{i}] = "))
                datos.append((x, y))
                break
            except ValueError:
                print("¡Error! Ingrese números válidos.")

    datos.sort(key=lambda p: p[0])
    datos_compartidos = datos.copy()
    return datos

def mostrar_tabla_previa(datos):
    """Muestra los datos capturados"""
    print("\nTabla de datos:")
    print("-"*40)
    print("| {:^4} | {:^15} | {:^15} |".format("i", "x_i", "y_i"))
    print("-"*40)
    for i, (x, y) in enumerate(datos):
        print("| {:^4} | {:^15.4f} | {:^15.4f} |".format(i, x, y))
    print("-"*40)

# 2.1 Diferencias Divididas
def diferencias_divididas():
    """Implementación de diferencias divididas"""
    if not datos_compartidos:
        print("\nPrimero debe capturar datos (Opción 2.1)")
        return

    xi, yi = zip(*datos_compartidos)
    xi, yi = list(xi), list(yi)
    n = len(xi)

    # Construir tabla de diferencias
    tabla = [yi.copy()]
    for i in range(1, n):
        fila_actual = []
        for j in range(n - i):
            diff = (tabla[i-1][j+1] - tabla[i-1][j]) / (xi[j+i] - xi[j])
            fila_actual.append(diff)
        tabla.append(fila_actual)

    # Mostrar tabla
    print("\nTabla de diferencias divididas:")
    headers = ["x", "f[x]"] + [f"f[{''.join(['x']*(k+1))}]" for k in range(1, n)]
    rows = []
    for i in range(n):
        row = [xi[i]]
        for j in range(len(tabla)):
            if j < len(tabla) and i < len(tabla[j]):
                row.append(tabla[j][i])
            else:
                row.append("")
        rows.append(row)

    print(tabulate(rows, headers=headers, tablefmt="grid"))

    # Interpolar
    try:
        x_interp = float(input("\nPunto a interpolar: "))
        if x_interp < min(xi) or x_interp > max(xi):
            print("¡Advertencia: El punto está fuera del rango de interpolación!")

        # Evaluar polinomio
        resultado = tabla[0][0]
        producto = 1.0
        for i in range(1, n):
            producto *= (x_interp - xi[i-1])
            resultado += tabla[i][0] * producto

        print(f"\nResultado de interpolación en x={x_interp}: {resultado:.6f}")

    except ValueError:
        print("Error: Ingrese un valor numérico válido.")

# 2.2 Spline Cúbico
def spline_cubico():
    """Implementación de splines cúbicos"""
    if not datos_compartidos:
        print("\nPrimero debe capturar datos (Opción 2.1)")
        return

    xi, yi = zip(*datos_compartidos)
    xi, yi = list(xi), list(yi)
    n = len(xi)

    # Implementación del spline cúbico natural
    h = [xi[i+1] - xi[i] for i in range(n-1)]

    # Construir matriz tridiagonal
    A = np.zeros((n, n))
    A[0, 0] = 1
    A[-1, -1] = 1

    for i in range(1, n-1):
        A[i, i-1] = h[i-1]
        A[i, i] = 2*(h[i-1] + h[i])
        A[i, i+1] = h[i]

    # Vector b
    b = np.zeros(n)
    for i in range(1, n-1):
        b[i] = 3*((yi[i+1]-yi[i])/h[i] - (yi[i]-yi[i-1])/h[i-1])

    # Resolver sistema
    c = np.linalg.solve(A, b)

    # Calcular coeficientes
    coeficientes = []
    for i in range(n-1):
        a = yi[i]
        b_i = (yi[i+1]-yi[i])/h[i] - h[i]*(2*c[i]+c[i+1])/3
        d = (c[i+1]-c[i])/(3*h[i])
        coeficientes.append((a, b_i, c[i], d))

    # Mostrar resultados
    print("\nCoeficientes de los splines cúbicos:")
    for i in range(n-1):
        print(f"\nIntervalo [{xi[i]:.2f}, {xi[i+1]:.2f}]:")
        print(f"S_{i}(x) = {yi[i]:.4f} + {coeficientes[i][1]:.4f}(x-{xi[i]:.2f}) + {coeficientes[i][2]:.4f}(x-{xi[i]:.2f})² + {coeficientes[i][3]:.4f}(x-{xi[i]:.2f})³")

    # Evaluar en un punto
    try:
        x_eval = float(input("\nPunto a evaluar: "))
        if x_eval < xi[0] or x_eval > xi[-1]:
            print("¡Advertencia: El punto está fuera del rango!")

        # Encontrar intervalo
        for i in range(n-1):
            if xi[i] <= x_eval <= xi[i+1]:
                dx = x_eval - xi[i]
                a, b_i, c_i, d = coeficientes[i]
                resultado = a + b_i*dx + c_i*dx**2 + d*dx**3
                print(f"\nResultado en x={x_eval}: {resultado:.6f}")
                break
    except ValueError:
        print("Error: Ingrese un valor numérico válido.")

def menu_interpolacion():
    """Submenú para interpolación y ajuste"""
    while True:
        print("\n=== Interpolación y Ajuste de Curvas ===")
        print("1. Leer tabla de datos")
        print("2. Interpolación (Diferencias divididas)")
        print("3. Ajuste por spline cúbico")
        print("4. Volver al menú principal")

        opcion = input("Seleccione una opción (1-4): ")

        if opcion == "1":
            datos = capturar_datos()
            if datos:
                mostrar_tabla_previa(datos)
        elif opcion == "2":
            diferencias_divididas()
        elif opcion == "3":
            spline_cubico()
        elif opcion == "4":
            break
        else:
            print("Opción no válida.")

# =============================================
# 3. Integración (datos equidistantes)
# =============================================
def f1(x):
    return (x**4 * math.sqrt(3 + 2 * x**2)) / 3

def f2(x):
    return x**5 / math.sqrt(x**2 + 4)

def romberg_integration(f, a, b, precision_digits):
    max_iter = 20
    R = [[0] * max_iter for _ in range(max_iter)]
    h = b - a
    R[0][0] = (f(a) + f(b)) * h / 2

    for i in range(1, max_iter):
        h /= 2
        subtotal = sum(f(a + (2 * k - 1) * h) for k in range(1, 2**(i-1) + 1))
        R[i][0] = R[i-1][0] / 2 + h * subtotal

        for j in range(1, i + 1):
            R[i][j] = (4**j * R[i][j-1] - R[i-1][j-1]) / (4**j - 1)

        if abs(R[i][i] - R[i-1][i-1]) < 10**(-precision_digits):
            return R[i][i], R[:i+1]

    return R[max_iter-1][max_iter-1], R

def print_romberg_table(R, precision_digits):
    print("\nTabla de Romberg:")
    for i, row in enumerate(R):
        print(f"Nivel {i}: ", end="")
        for val in row[:i+1]:
            print(f"{val:.{precision_digits}f}", end="\t")
        print()

def ejecutar_integracion():
    funciones = {
        "1": ("f(x) = x⁴ √(3 + 2x²) / 3", f1),
        "2": ("f(x) = x⁵ / √(x² + 4)", f2),
    }

    print("\n=== Integración Numérica por Método de Romberg ===")
    print("Desarrollado por:")
    print("- Martinez Martinez Andres A.")
    print("- Lopez Blancas Tania")
    print("- Gomez Luna Paola Michelle")
    print("\nFunciones disponibles:")
    for key, (desc, _) in funciones.items():
        print(f"{key}. {desc}")
    print("3. Volver al menú principal")

    while True:
        opcion = input("\nSeleccione una función (1-2) o 3 para volver: ")
        if opcion == "3":
            return

        if opcion not in funciones:
            print("Opción inválida.")
            continue

        try:
            a = float(input("Límite inferior (a): "))
            b = float(input("Límite superior (b): "))
            if a >= b:
                print("Error: a debe ser menor que b")
                continue

            precision = int(input("Dígitos de precisión: "))
            if precision < 1:
                print("Error: La precisión debe ser ≥1")
                continue

            funcion = funciones[opcion][1]
            resultado, tabla = romberg_integration(funcion, a, b, precision)

            print(f"\nResultado: ∫ f(x) dx ≈ {resultado:.{precision}f}")
            print_romberg_table(tabla, precision)

        except ValueError:
            print("Error: Entrada inválida")

        if input("\n¿Desea realizar otra integración? (s/n): ").lower() != 's':

            break

# =============================================
# Menú Principal
# =============================================

def mostrar_banner():
    print("""
*******************************************************
      * MÉTODOS NUMÉRICOS AVANZADOS *
      Desarrollado por:
      - López Blancas Tania
      - Martínez Martínez Andrés A.
      - Gómez Luna Paola Michelle
*******************************************************
""")

def main():
    mostrar_banner()

    while True:
        print("\nMENÚ PRINCIPAL:")
        print("1. Sistemas de ecuaciones no lineales (Broyden)")
        print("2. Interpolación y ajuste de curvas")
        print("3. Integración Numérica (Romberg)")
        print("4. Salir")

        opcion = input("Seleccione una opción (1-4): ")

        if opcion == "1":
            ejecutar_broyden()
        elif opcion == "2":
            menu_interpolacion()
        elif opcion == "3":
            ejecutar_integracion()
        elif opcion == "4":
            print("\n¡Gracias por usar el programa!")
            break
        else:
            print("Opción no válida. Intente nuevamente.")

if __name__ == "__main__":
    main()


*******************************************************
      * MÉTODOS NUMÉRICOS AVANZADOS *
      Desarrollado por:
      - López Blancas Tania
      - Martínez Martínez Andrés A.
      - Gómez Luna Paola Michelle
*******************************************************


MENÚ PRINCIPAL:
1. Sistemas de ecuaciones no lineales (Broyden)
2. Interpolación y ajuste de curvas
3. Integración Numérica (Romberg)
4. Salir
Seleccione una opción (1-4): 3

=== Integración Numérica por Método de Romberg ===
Desarrollado por:
- Martinez Martinez Andres A.
- Lopez Blancas Tania
- Gomez Luna Paola Michelle

Funciones disponibles:
1. f(x) = x⁴ √(3 + 2x²) / 3
2. f(x) = x⁵ / √(x² + 4)
3. Volver al menú principal

Seleccione una función (1-2) o 3 para volver: 1
Límite inferior (a): 0
Límite superior (b): 9
Dígitos de precisión: 5

Resultado: ∫ f(x) dx ≈ 42328.72107

Tabla de Romberg:
Nivel 0: 126416.35642	
Nivel 1: 67265.00012	47547.88135	
Nivel 2: 48807.41962	42654.89279	42328.69355	
Nivel 3: 43963.6832